In [20]:
#%pip install tensorflow
#%pip install seaborn
#%pip install sklearn

In [41]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [42]:
df = pd.read_csv('./data/diabetes_012_health_indicators_BRFSS2015.csv')

PREPARAÇÃO DO DATASET EM TREINO E TESTE

In [43]:
x_data = df.drop(["Diabetes_012"],axis=1)
y_data = df["Diabetes_012"]
MinMaxScaler = preprocessing.MinMaxScaler()
X_data_minmax = MinMaxScaler.fit_transform(x_data)
data = pd.DataFrame(X_data_minmax,columns=x_data.columns)

X_train, X_test, y_train, y_test = train_test_split(data, y_data,test_size=0.2, random_state = 1)

In [49]:
print(df[df["Diabetes_012"] == 0].shape[0])
print(df[df["Diabetes_012"] == 1].shape[0])
print(df[df["Diabetes_012"] == 2].shape[0])

213703
4631
35346


In [52]:
from sklearn.utils import resample

df_class_0 = df[df["Diabetes_012"] == 0]
df_class_12 = df[df["Diabetes_012"] != 0]
# Downsample majority class
df_majority_downsampled = resample(df_class_0, 
                                 replace=False,    # sample without replacement
                                 n_samples=45000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_class_12])
 
# Display new class counts
print(df_downsampled[df_downsampled["Diabetes_012"] == 0].shape[0])
print(df_downsampled[df_downsampled["Diabetes_012"] == 1].shape[0])
print(df_downsampled[df_downsampled["Diabetes_012"] == 2].shape[0])

213703
4631
35346


APLICAÇÃO DO MODELO COM K-NEAREST-NEIGHBORS

In [17]:
knn_clf=KNeighborsClassifier(n_neighbors=8)
knn_clf.fit(X_train,y_train)
ypred=knn_clf.predict(X_test)

In [18]:
result1 = classification_report(y_test, ypred)
print("Classification Report: ")
print (result1)
result2 = accuracy_score(y_test,ypred)
print("Accuracy:",result2)

Classification Report: 
              precision    recall  f1-score   support

         0.0       0.86      0.97      0.91     42798
         1.0       0.00      0.00      0.00       873
         2.0       0.41      0.12      0.19      7065

    accuracy                           0.84     50736
   macro avg       0.42      0.37      0.37     50736
weighted avg       0.78      0.84      0.79     50736

Accuracy: 0.837038789025544


APLICAÇÃO DO MODELO REDES NEURAIS

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [44]:
model = Sequential()
model.add(Dense(100, activation="relu", input_shape=(len(x_data.columns),)))
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 100)               2200      
                                                                 
 dense_14 (Dense)            (None, 100)               10100     
                                                                 
 dense_15 (Dense)            (None, 3)                 303       
                                                                 
Total params: 12,603
Trainable params: 12,603
Non-trainable params: 0
_________________________________________________________________


TypeError: Invalid keyword argument(s) in `compile()`: ({'class_weight'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [36]:
y_pred = model.predict(X_test)

1586/1586 [==============================] - 7s 5ms/step
              precision    recall  f1-score   support

         0.0       0.85      0.99      0.92     42798
         1.0       0.00      0.00      0.00       873
         2.0       0.57      0.09      0.16      7065

    accuracy                           0.85     50736
   macro avg       0.47      0.36      0.36     50736
weighted avg       0.80      0.85      0.80     50736



AttributeError: 'numpy.ndarray' object has no attribute 'values_count'

In [40]:
predicted_categories = np.argmax(y_pred, axis = 1)
print(classification_report(y_test, predicted_categories, zero_division=0))

print("Class 0 predictions: ", np.count_nonzero(predicted_categories == 0))
print("Class 1 predictions: ", np.count_nonzero(predicted_categories == 1))
print("Class 2 predictions: ", np.count_nonzero(predicted_categories == 2))


              precision    recall  f1-score   support

         0.0       0.85      0.99      0.92     42798
         1.0       0.00      0.00      0.00       873
         2.0       0.57      0.09      0.16      7065

    accuracy                           0.85     50736
   macro avg       0.47      0.36      0.36     50736
weighted avg       0.80      0.85      0.80     50736

Class 0 predictions:  49588
Class 1 predictions:  0
Class 2 predictions:  1148
